#  DDP

### 1. running on our DGX
- docker run -it --rm --gpus '"device=4,5,6,7"' -p 12344:12344 --name msalehjahromi --shm-size=192G  --user $(id -u):$(id -g) --group-add 1944259512 --cpuset-cpus=49-96 -v /rsrch7/home/ip_rsrch/wulab/:/rsrch7/home/ip_rsrch/wulab -v /rsrch1/ip/msalehjahromi/:/rsrch1/ip/msalehjahromi --name mori_jupyter nnunetv2:msalehjahromi
- cd /rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/n_GPU_A100
- jupyter notebook --ip 0.0.0.0 --port 12344
- http://1mcprddgx05

In [3]:
!gpustat

2a974e97ed31              Wed May 28 17:59:21 2025  535.216.01
[0] NVIDIA A100-SXM4-40GB | 37'C,  77 % | 26949 / 40960 MB |
[1] NVIDIA A100-SXM4-40GB | 41'C,  64 % | 26203 / 40960 MB |
[2] NVIDIA A100-SXM4-40GB | 27'C,   0 % |   628 / 40960 MB |
[3] NVIDIA A100-SXM4-40GB | 29'C,   0 % |   628 / 40960 MB |


In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

import torch
print(torch.cuda.device_count())   # → 2
print(torch.cuda.current_device()) # → 0 (this maps to your original GPU 2)


2
0


### 2. Probably need installation, restart after and run 3

In [5]:
# ## !pip uninstall -q histolab -y
# # Install required packages first
# import os
# import subprocess
# pip_commands = [
#     ["pip", "install","-q", "--extra-index-url", "https://download.pytorch.org/whl/cu117", 
#      "torch==2.0.0", "torchvision==0.15.0", "omegaconf", "torchmetrics==0.10.3", 
#      "fvcore", "iopath", "xformers==0.0.18", "submitit", "numpy<2.0"],
#     ["pip", "install", "-q",  "--extra-index-url", "https://pypi.nvidia.com", "cuml-cu11"],
#     ["pip", "install","-q",  "black==22.6.0", "flake8==5.0.4", "pylint==2.15.0"],
#     ["pip", "install", "-q", "mmsegmentation==0.27.0"],
#     ["pip", "install","-q", "mmcv-full==1.5.0"],
#     ["pip", "install","-q", "nibabel"]
# ]

# for cmd in pip_commands:
#     try:
#         print(cmd)
#         subprocess.run(cmd, check=True)
#     except subprocess.CalledProcessError as e:
#         print(f"Failed to install packages with command: {cmd}")
#         print(f"Error: {e}")

### 3. running, notice important hyperparams
-  which model are use so vmin -1000., vmax +150, eps 0.00005
- `num-workers`
- `epochs`: 100
- `accum-steps`: 2000 ?
- `max-chunks`: 66 
- `lr`:  lr*0.1 on base ?
- `warmup-steps`: 5k
- `print-every` : 5000
- `val-every` : 40000

In [ ]:
## Install required packages first

import os
import subprocess
# Set required environment variables
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29500"
os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "2"
os.environ["LOCAL_RANK"] = "0"
os.environ["LOCAL_WORLD_SIZE"] = "2"

# Build the command with --install-packages flag removed
command = [
    "python3",
    "/rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/n_GPU_A100_v2/2_run_fineTune_ddp_launcher.py", ##changed
    "--num-gpus", "2",
    "--csv", "/rsrch1/ip/msalehjahromi/codes/FineTune/nlst_event_train_val_.csv",
    "--accum-steps", "500", ###############
    "--num-workers", "5",
    "--epochs", "200", ###############
    "--lr", "0.0004", ###############
    "--weight-decay", "0.0001",
    "--optimizer", "adamw",
    "--num-attn-heads", "3",
    "--num-layers", "2",
    "--dropout", "0.35",
    "--unfreeze-strategy", "all",  
    "--output", "/rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/n_GPU_A100_v2/output_ddp_p16_23_1150_ac2000",
    "--print-every", "50", ################ 2000
    "--val-every", "10000", ################ 50k
    "--metrics-dir", "/rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/metrics_multi_gpu"
]

# Run the command
try:
    subprocess.run(command, check=True)
except subprocess.CalledProcessError as e:
    print(f"Error occurred: {e}")
    print(f"Command output: {e.output if hasattr(e, 'output') else 'No output available'}")

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
/rsrch1/ip/msalehjahromi/codes/dinov2-torchrun-dataloader6/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/rsrch1/ip/msalehjahromi/codes/dinov2-torchrun-dataloader6/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/rsrch1/ip/msalehjahromi/codes/dinov2-torchrun-dataloader6/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")
/rsrch1/ip/msalehjahromi/codes/dinov2-torchrun-dataloader6/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("x

Files in the directory: ['n_GPU_A100_v2', '._.gitignore', '._.DS_Store', '._See_attention.ipynb', '1_GPU_H100', 'See_attention.ipynb', 'n_GPU_A100', 'metrics_single_gpu', 'n_H100onH100', '.DS_Store', '1_GPU_A100', 'ReadMe.md', '.gitignore', 'metrics_multi_gpu', '.git']
Files in the directory: ['n_GPU_A100_v2', '._.gitignore', '._.DS_Store', '._See_attention.ipynb', '1_GPU_H100', 'See_attention.ipynb', 'n_GPU_A100', 'metrics_single_gpu', 'n_H100onH100', '.DS_Store', '1_GPU_A100', 'ReadMe.md', '.gitignore', 'metrics_multi_gpu', '.git']


2025-05-28 17:59:41,333 - INFO - Starting training on 2 GPUs with full model copies (DDP)


Train dataset created on rank 
Train dataset created on rank 

==== Dataset Statistics ====
Total samples: 43852
Training samples: 38382
Validation samples: 5470

1-year-cancer:
  Train positive: 1428 (3.72%)
  Val positive: 188 (3.44%)

2-year-cancer:
  Train positive: 1891 (4.93%)
  Val positive: 256 (4.68%)

3-year-cancer:
  Train positive: 2282 (5.95%)
  Val positive: 315 (5.76%)

4-year-cancer:
  Train positive: 2586 (6.74%)
  Val positive: 363 (6.64%)

5-year-cancer:
  Train positive: 2838 (7.39%)
  Val positive: 403 (7.37%)

6-year-cancer:
  Train positive: 2988 (7.78%)
  Val positive: 429 (7.84%)

Total validation samples: 5470
Created distributed validation loader, each rank processes ~2568 samples


2025-05-28 17:59:43,249 - INFO - using MLP layer as FFN


...

2025-05-28 19:38:50,983 - INFO - iteration: 2850 | epoch: 0 | loss: 1.0081 | lr: 0.0000 | type: training | acc_task0: 0.4937 | sens_task0: 0.0000 | spec_task0: 1.0000 | acc_task1: 0.5193 | sens_task1: 0.9986 | spec_task1: 0.0007 | acc_task2: 0.4705 | sens_task2: 0.0000 | spec_task2: 1.0000 | acc_task3: 0.5408 | sens_task3: 0.7906 | spec_task3: 0.2488 | acc_task4: 0.6159 | sens_task4: 1.0000 | spec_task4: 0.0011 | acc_task5: 0.7445 | sens_task5: 1.0000 | spec_task5: 0.0000



Step 2900 (Epoch 1):
Loss: 0.9448
Learning rate: 0.000016
Accuracy:     T0:0.49  T1:0.52  T2:0.47  T3:0.54  T4:0.62  T5:0.75  
Sensitivity:  T0:0.00  T1:1.00  T2:0.00  T3:0.78  T4:1.00  T5:1.00  
Specificity:  T0:1.00  T1:0.00  T2:1.00  T3:0.26  T4:0.00  T5:0.00  


2025-05-28 19:41:01,221 - INFO - iteration: 2900 | epoch: 0 | loss: 0.9448 | lr: 0.0000 | type: training | acc_task0: 0.4924 | sens_task0: 0.0000 | spec_task0: 1.0000 | acc_task1: 0.5207 | sens_task1: 0.9987 | spec_task1: 0.0007 | acc_task2: 0.4692 | sens_task2: 0.0000 | spec_task2: 1.0000 | acc_task3: 0.5407 | sens_task3: 0.7815 | spec_task3: 0.2576 | acc_task4: 0.6168 | sens_task4: 1.0000 | spec_task4: 0.0011 | acc_task5: 0.7465 | sens_task5: 1.0000 | spec_task5: 0.0000



Step 2950 (Epoch 1):
Loss: 1.0869
Learning rate: 0.000016
Accuracy:     T0:0.49  T1:0.52  T2:0.47  T3:0.54  T4:0.62  T5:0.75  
Sensitivity:  T0:0.00  T1:1.00  T2:0.00  T3:0.77  T4:1.00  T5:1.00  
Specificity:  T0:1.00  T1:0.00  T2:1.00  T3:0.27  T4:0.00  T5:0.00  


2025-05-28 19:42:52,545 - INFO - iteration: 2950 | epoch: 0 | loss: 1.0869 | lr: 0.0000 | type: training | acc_task0: 0.4922 | sens_task0: 0.0000 | spec_task0: 1.0000 | acc_task1: 0.5207 | sens_task1: 0.9987 | spec_task1: 0.0007 | acc_task2: 0.4690 | sens_task2: 0.0000 | spec_task2: 1.0000 | acc_task3: 0.5398 | sens_task3: 0.7729 | spec_task3: 0.2656 | acc_task4: 0.6164 | sens_task4: 1.0000 | spec_task4: 0.0010 | acc_task5: 0.7465 | sens_task5: 1.0000 | spec_task5: 0.0000
